# Build a simple ML pipeline for image classification

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` with components

**Motivations** -This tutorial shows how to train a simple deep neural network using the [Fashion MNIST dataset and Keras on Azure Machine Learning. Fashion-MNIST is a dataset of Zalando's article imagesâ€”consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.


# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [22]:
#import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import load_component

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [23]:
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    # If exception happens when retrieve token, try exclude the failed credential like this then try again:
    # Exclude VSCode credential:
    # credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    raise Exception("Failed to retrieve a token from the included credentials due to the following exception, try to add `exclude_xxx_credential=True` to `DefaultAzureCredential` and try again.") from ex

## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace.

In [24]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "<SUBSCRIPTION_ID>",
        "resource_group": "<RESOURCE_GROUP>",
        "workspace_name": "<WORKSPACE_NAME>"
    }

    if client_config["subscription_id"].startswith('<'):
        print("please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <WORKSPACE_NAME> in notebook cell")
        raise ex
    else:  # write and reload from config file
        import json, os
        config_path = "../../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: D:\repro\azureml-examples\sdk\jobs\.azureml\config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001BBF95EAC10>,
         subscription_id=4faaaf21-663f-4391-96fd-47197c630979,
         resource_group_name=designertestrg,
         workspace_name=DesignerTest-centraluseuap)


## 1.4. Retrieve or create an Azure Machine Learning compute target

In [25]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    ml_client.compute.get(name=cluster_name)
except Exception:
    print('Creating a new compute target...')
    from azure.ml.entities import AmlCompute
    compute = AmlCompute(
        name=cluster_name,
        size="STANDARD_NC6",
        max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute)

## 1.5. Prepare dataset
By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

In [26]:
from azure.ml.entities import Data, JobInput
from azure.ml._constants import AssetTypes, InputOutputModes


fashion_ds = JobInput(path="https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion")

In [27]:
fashion_ds

{'type': 'uri_folder', 'path': 'https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion', 'mode': 'ro_mount'}

# 2. Define command component via YAML
Below is a basic example to define command component  using YAML.


In [28]:
prepare_dataset = load_component(yaml_file='./keras-mnist-fashion/prepare.yaml')
train_model = load_component(yaml_file='./keras-mnist-fashion/train.yaml')

# 3. Basic pipeline job

## 3.1 Build pipeline

In step one, we will load the image and labels from Fashion MNIST dataset into mnist_train.csv and mnist_test.csv. Then in step two, train CNN with Keras

In [29]:
# define pipeline
@dsl.pipeline(
    name='image_classification_keras_minist_convnet',
    description='E2E image classification pipeline with keras',
    default_compute=cluster_name,
)
def image_classification_keras_minist_convnet():
    prepare_dataset_node = prepare_dataset(input_path=fashion_ds)

    train_node = train_model(input_path=prepare_dataset_node.outputs.output_path)
    
    # Return: pipeline outputs
    return {
        "trained_model": train_node.outputs.output_model,
    }



# create a pipeline
pipeline = image_classification_keras_minist_convnet()

In [30]:
print(pipeline)

name: khaki_chayote_t5hp66tqp6
display_name: image_classification_keras_minist_convnet
description: E2E image classification pipeline with keras
type: pipeline
inputs: {}
outputs:
  trained_model: null
tags: {}
compute: azureml:cpu-cluster
jobs:
  prepare_dataset_node:
    $schema: '{}'
    type: command
    inputs:
      input_path:
        mode: ro_mount
        type: uri_folder
        path: azureml:https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion
    outputs: {}
    command: ''
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: convert_to_csvfile
      version: 0.0.1
      display_name: Convert to CSV File
      description: Convert dataset to csv file.
      type: command
      inputs:
        input_path:
          type: uri_folder
      outputs:
        output_path:
          type: uri_folder
      command: python prepare.py --mounted_input_path ${{inputs.input_path}} --mounted_output_path
     

# 3.2 Submit pipeline job

In [31]:
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name='pipeline_samples')
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_samples,khaki_chayote_t5hp66tqp6,pipeline,Preparing,Link to Azure Machine Learning studio


In [32]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: khaki_chayote_t5hp66tqp6
Web View: https://ml.azure.com/runs/khaki_chayote_t5hp66tqp6?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/designertestrg/workspaces/DesignerTest-centraluseuap

Streaming logs/azureml/executionlogs.txt

[2022-04-08 02:39:46Z] Submitting 1 runs, first five are: aec24da9:cefad0ae-d57a-42e7-a3b0-37d06a80e733

Execution Summary
RunId: khaki_chayote_t5hp66tqp6
Web View: https://ml.azure.com/runs/khaki_chayote_t5hp66tqp6?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/designertestrg/workspaces/DesignerTest-centraluseuap


Exception: Exception : 
 "Detailed error not set on the Run. Please check the logs for details." 

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)